In [1]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [2]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [3]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [4]:
# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [5]:
xq.shape

(1, 128)

In [6]:
xb.shape

(1000000, 128)

In [7]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

In [9]:
d = 128  # dimensionality of Sift1M data
k = 10  # number of nearest neighbors to return

import faiss

index = faiss.IndexFlatIP(d)
index.add(xb)



In [10]:
%%time
D, I = index.search(xq, k)

CPU times: user 17.8 ms, sys: 0 ns, total: 17.8 ms
Wall time: 17.6 ms


In [21]:
%%time
# LHS
nbits = d*4  # resolution of bucketed vectors
# initialize index and add vectors
index = faiss.IndexLSH(d, nbits)
index.add(xb)


In [22]:
%%time
# and search
D, I = index.search(xq, k)

CPU times: user 1.97 s, sys: 0 ns, total: 1.97 s
Wall time: 72.1 ms


In [23]:
%%time
# HNSWFlat
M = 64 # number of connections each vertex will have
ef_search = 32 # depth of layers explored during search
ef_construction = 64 # depth of layers explored during index construction
# initialize index (d == 128)
index = faiss.IndexHNSWFlat(d, M)

# set efConstruction and efSearch parameters
index.hnsw.efConstruction = ef_construction
index.hnsw.efSearch = ef_search 

# add data to index
index.add(xb)

In [24]:
%%time
# and search
D, I = index.search(xq, k)

CPU times: user 293 ms, sys: 0 ns, total: 293 ms
Wall time: 12.7 ms


In [26]:
%%time
# IVF Implementation
nlist = 128  # number of cells/clusters to partition data into
quantizer = faiss.IndexFlatIP(d)  # how the vectors will be stored/compared
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.train(xb)  # we must train the index to cluster into cells
index.add(xb)

index.nprobe = 8  # set how many of nearest cells to search

CPU times: user 4min 38s, sys: 2min 29s, total: 7min 7s
Wall time: 13.6 s


In [27]:
%%time
# and search
D, I = index.search(xq, k)

CPU times: user 1.69 ms, sys: 1.83 ms, total: 3.52 ms
Wall time: 3.14 ms
